In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import requests
import time
import re
import numpy as np
import json

In [18]:
%%time
df_final = pd.DataFrame()
for i in range(0, 49): #2074
    if i == 0:
        URL = 'https://www.vivanuncios.com.mx/s-departamentos-en-venta/roma-norte/v1c1294l13669p1'
    else:
        URL = f'https://www.vivanuncios.com.mx/s-departamentos-en-venta/roma-norte/page-{i+1}/v1c1294l13669p{i+1}'
    
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, 'html.parser')
    results = soup.find_all("div", {"class": "tileV2"})
    
    
    links = list()
    for i in range(0, len(results)):
        link = results[i].find_all("a", {"class": "href-link tile-title-text"})[0]['href']
        links.append(link)
        
    temp = pd.DataFrame(links, columns=['link'])
    
    df_final = pd.concat([df_final, temp])
    time.sleep(1)


CPU times: user 6.16 s, sys: 254 ms, total: 6.42 s
Wall time: 1min 55s


In [19]:
df_final.to_csv('../data/raw/scrapper_viva_main.csv', index=False)

In [21]:
df_final = df_final.reset_index(drop=True)

In [22]:
df_final['id'] = 1
df_final['id'] = df_final['id'].cumsum()

In [23]:
df_final.head(1)

,link,id
0,/d-desarrollo-prisma/5bfecd44e4b09ae2809ee141,1


In [77]:
df_final.head(1)

,link,id
0,/d-desarrollo-prisma/5bfecd44e4b09ae2809ee141,1


In [83]:
df_final['id'][i]

1

In [134]:
len(df_final)

1103

In [188]:
%%time
attributes_table = pd.DataFrame()
characteristics = pd.DataFrame()
for i in range(0, len(df_final)): #len(df_final)
    try:
        URL = 'https://www.vivanuncios.com.mx' + df_final['link'][i]
        page = requests.get(URL)
        soup = BeautifulSoup(page.content, 'html.parser')
        
        container = soup.find_all("div", {"class": "category"})

        for j in range(0, len(container)):

            attribute_name = container[j].find_all("span", "pri-props-name")[0].get_text()
            attribute_value = container[j].find_all("span", "pri-props-value")[0].get_text()

            att_tmp = pd.DataFrame([df_final['id'][i]], columns=['id'])
            att_tmp['attribute_name'] = attribute_name
            att_tmp['attribute_value'] = attribute_value

            attributes_table = pd.concat([attributes_table,att_tmp])


        char_tmp = pd.DataFrame([df_final['id'][i]], columns=['id'])
        
        aa = soup.select_one("script[type='application/ld+json']")
        js = json.loads(aa.text) 
        try:
            price = js[0]['offers']['price']
            char_tmp['price'] = price
        except: char_tmp['price'] = np.nan
        
        try:
            name = js[0]['name']
            char_tmp['name'] = name
        except: char_tmp['name'] = np.nan
        
        try:
            lat = js[0]['geo']['latitude']
            lon = js[0]['geo']['longitude']
            char_tmp['lat'] = lat
            char_tmp['lon'] = lon
        except: 
            char_tmp['lat'] = np.nan
            char_tmp['lon'] = np.nan

        characteristics = pd.concat([characteristics,char_tmp])
        time.sleep(1)
    except: None

CPU times: user 2min 2s, sys: 3.85 s, total: 2min 6s
Wall time: 30min 25s


In [191]:
characteristics.to_csv('../data/raw/scrapper_viva_chr.csv', index=False)
attributes_table.to_csv('../data/raw/scrapper_viva_att.csv', index=False)

# JOIN

In [193]:
table = pd.pivot_table(attributes_table, values='attribute_value', index=['id'],
                    columns=['attribute_name'], 
                       aggfunc=lambda x: ' '.join(x)).reset_index()

In [194]:
table = table[['id', 'Baños:', 'Metros Cuadrados:', 'Recámara(s):',
               'Superficie Total:', 'Terreno:'
              ]]

In [195]:
table = table.rename(columns={'Baños:':'Baños',
                      'Metros Cuadrados:':'m2_totales',
                      'Recámara(s):':'Recámaras',
                      'Superficie Total:':'superficie',
                      'Terreno:':'terreno',
                     })

In [196]:
len(characteristics)

730

In [246]:
characteristics_f = characteristics.merge(table, how='left', on='id')

In [247]:
len(characteristics_f)

730

In [248]:
characteristics_f.isna().sum()

id              0
price           0
name            0
lat             0
lon             0
Baños           4
m2_totales    730
Recámaras       0
superficie    730
terreno       342
dtype: int64

In [249]:
characteristics_f['Recámaras'] = characteristics_f['Recámaras'].apply(lambda x: re.sub("[^0-9]", "", str(x)))
characteristics_f['Recámaras'] = characteristics_f['Recámaras'].astype(int)

In [250]:
characteristics_f['Baños'] = characteristics_f['Baños'].apply(lambda x: re.sub("[^0-9]", "", str(x)))
characteristics_f.loc[characteristics_f['Baños']=="", 'Baños'] = np.nan

In [251]:
characteristics_f['Baños'] = characteristics_f['Baños'].fillna(-1)
characteristics_f['Baños'] = characteristics_f['Baños'].astype(int)

In [252]:
characteristics_f.loc[characteristics_f['Baños']==-1, 'Baños'] = np.nan

In [253]:
characteristics_f['terreno'] = characteristics_f['terreno'].apply(lambda x: str(x).split('m')[0])
characteristics_f.loc[characteristics_f['terreno']=="nan", 'terreno'] = -1
characteristics_f['terreno'] = characteristics_f['terreno'].astype(int)

In [254]:
characteristics_f.loc[characteristics_f['terreno']==-1, 'terreno'] = np.nan

In [255]:
characteristics_f[['price', 'Recámaras', 'Baños', 'terreno']].corr()

,price,Recámaras,Baños,terreno
price,1.000000,0.246131,0.475879,0.596648
Recámaras,0.246131,1.000000,0.618709,0.607578
Baños,0.475879,0.618709,1.000000,0.571975
terreno,0.596648,0.607578,0.571975,1.000000


In [256]:
characteristics_f = characteristics_f.drop(columns=['m2_totales', 'superficie'])

In [257]:
characteristics_f.sample()

,id,price,name,lat,lon,Baños,Recámaras,terreno
410,668,3425000,"Departamento en Venta en Roma Norte, Cuahutemoc",19.42156,-99.17041,1.0,1,NaN


In [258]:
characteristics_f = characteristics_f.drop_duplicates(subset=['name',
                                                             'lat', 'lon'
                                                             ])

In [259]:
len(characteristics_f)

635

In [261]:
characteristics_f.to_csv('../data/processed/scrapper_viva_data.csv', index=False)